In [16]:
from skimage import io
from skimage import feature
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

source = '/home/gc1569/Capstone/Cropped_Images/'
feature_list = []

for i, pic in enumerate([f for f in os.listdir(source) if f.endswith('.jpg')]):
    feature_dict = {}
    image = io.imread(source + pic, as_grey=True)
    feature_dict['lbp'] = np.array(feature.local_binary_pattern(image, 10, 10)).flatten()
    feature_dict['fast'] = np.array(feature.corner_fast(image, n=12, threshold=0.15)).flatten()
    feature_dict['shape_index'] = np.array(feature.shape_index(image, sigma=1, mode='constant', cval=0)).flatten()
    HX, HY, HZ = feature.hessian_matrix(image, sigma=1, mode='constant', cval=0, order=None)
    feature_dict['hessian'] = np.array(feature.hessian_matrix_eigvals(HX, HY, HZ)[0]).flatten()
    HX, HY, HZ = feature.structure_tensor(image, sigma=1, mode='constant', cval=0)
    feature_dict['structure'] = np.array(feature.structure_tensor_eigvals(HX, HY, HZ)[0]).flatten()
    feature_dict['canny'] = 1*np.array(feature.canny(image, sigma=1.0, low_threshold=None, high_threshold=None, mask=None, use_quantiles=False)).flatten()
    feature_list.append(feature_dict)
    if (i%50 == 0):
        print('50', end=' ')

df = pd.DataFrame(feature_list)
df.to_csv('/home/gc1569/Capstone/Image_Features/features.csv', sep='\t')
print('All features extracted and saved in a csv file')

/home/gc1569/anaconda3/lib/python3.6/site-packages/skimage/feature/corner.py:166: UserWarning: deprecation warning: the default order of the hessian matrix values will be "row-column" instead of "xy" starting in skimage version 0.15. Use order="rc" or order="xy" to set this explicitly
  warn('deprecation warning: the default order of the hessian matrix values '


50 

/home/gc1569/anaconda3/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: invalid value encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/gc1569/anaconda3/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: divide by zero encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))


50 50 50 50 50 50 50 50 50 50 All features extracted and saved in a csv file


In [17]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pandas as pd

# #############################################################################
# Compute DBSCAN

db = DBSCAN(eps=0.3, min_samples=10).fit(df.values.tolist())
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

ValueError: setting an array element with a sequence.

In [19]:
for i in range(50):
    print(len(df['canny'][i]))

33124
123904
57600
65025
68644
51076
40804
126736
35721
39204
42436
97344
39204
123904
38416
25600
109561
38025
68644
46225
50176
62001
47089
63001
80656
44944
35721
30276
65025
15876
77841
87025
30625
80089
75076
47524
51984
63001
4624
82944
18496
108241
21025
84100
87616
71824
144400
42436
44521
30976
